# Formeln vom Manual

In [60]:
import numpy as np
import uncertainties as u
import uncertainties.unumpy as unp
import scipy.constants
import os
import pandas # pip install Jinja2
import pint # pip install pint OR conda install pint
units = pint.UnitRegistry()
umean = u.wrap(np.mean)
ustd = u.wrap(np.std)
import warnings
warnings.simplefilter("ignore")

def strip_units(np_arr):
    return np.array([a.to_base_units().magnitude for a in np_arr])

def extr_values(np_arr):
    return unp.nominal_values(np_arr)

# Formel (2)
# n'2(nu) = 1 + A/(nu0dash^2-nu^2)
# nu0dash, A sind Materialkonstanten
def n_squared_of_nu(nu, nu0dash, A):
    n_sq = 1 + A/(nu0dash**2-nu**2)
    return n_sq

# Formel (2) und die Wurzel gezogen
def n_of_nu(nu, nu0dash, A):
    n = np.sqrt(n_squared_of_nu(nu, nu0dash,A))
    return n

# First Task: Determine $\varphi$
Measure both positions where the reflections of the slit image are observed: $A_{1}$, $A_{2}$. Then $\beta = A_{1} - A_{2}$. Make sure the signs are correct (can we take the absolute value?). Do this three times, so we get three $\beta$ values. Take the mean of $\beta$. Then $\varphi = \pi - \beta$.

In [70]:
from numpy import extract


A1_meas, A2_meas = np.genfromtxt(os.path.join(os.getcwd(), "task1_measure_phi_data.csv"), delimiter=',', skip_header=1, unpack=True)

A_meas_uncertainty_guess = 99.999 # TODO change this value
A_unit = units.rad
A1s = np.array([u.ufloat(a1, A_meas_uncertainty_guess) for a1 in A1_meas]) *A_unit
A2s = np.array([u.ufloat(a2, A_meas_uncertainty_guess) for a2 in A2_meas]) *A_unit

Betas = A1s - A2s
Betas_Std = np.std(extr_values(Betas))

Beta = umean(Betas)

# PRINT AND GENERATE LATEX
DF_TABLE_1 = pandas.DataFrame({"A1 [%s]"%(units.rad): A1s, "A2 []": A2s, r'$\beta$': Betas})
print("Table of Data:")
print(DF_TABLE_1, '\n')
print("Table latex:")
print(DF_TABLE_1.style.to_latex(column_format='|l|l|l|l|'), '\n')
print("Mean(beta)+=prop_unc_a: ", Beta, '\n')
print("Mean(beta)+=Std(betas):", u.ufloat(Beta.nominal_value,Betas_Std)*A_unit)

Beta_Result = Beta
if (Beta.std_dev < Betas_Std):
    Beta_Result = u.ufloat(Beta.nominal_value, Betas_Std)*A_unit
    print("Standard deviation of the betas was higher so this was taken instead of the propagation of the errors of measurement of A.")
else:
    Beta_Result = Beta
    print("Standard deviation of the betas was lower than the propagation of the errors of measurement of A. So the propagated error was taken.")

print("Beta End Result: ", Beta_Result)

Phi_Result = scipy.constants.pi*A_unit - Beta

print("Phi end result:", Phi_Result)




Table of Data:
       A1 [radian]            A2 []           $\beta$
0  (5.6+/-1.0)e+02  (7.8+/-1.0)e+02  (-2.1+/-1.4)e+02
1  (5.3+/-1.0)e+02  (9.8+/-1.0)e+02  (-4.4+/-1.4)e+02
2  (5.5+/-1.0)e+02  (8.8+/-1.0)e+02  (-3.3+/-1.4)e+02 

Table latex:
\begin{tabular}{|l|l|l|l|}
 & A1 [radian] & A2 [] & $\beta$ \\
0 & (5.6+/-1.0)e+02 & (7.8+/-1.0)e+02 & (-2.1+/-1.4)e+02 \\
1 & (5.3+/-1.0)e+02 & (9.8+/-1.0)e+02 & (-4.4+/-1.4)e+02 \\
2 & (5.5+/-1.0)e+02 & (8.8+/-1.0)e+02 & (-3.3+/-1.4)e+02 \\
\end{tabular}
 

Mean(beta)+=prop_unc_a:  (-3.3+/-0.8)e+02 radian 

Mean(beta)+=Std(betas): (-3.3+/-0.9)e+02
Standard deviation of the betas was higher so this was taken instead of the propagation of the errors of measurement of A.
Beta End Result:  (-3.3+/-0.9)e+02 radian
Phi end result: (3.3+/-0.8)e+02 radian
